In [38]:
from keras import backend as K
from keras.layers import Input, Dense, Conv2D, MaxPooling2D
from keras.layers.core import Activation, Lambda, Dropout 
from keras.layers.core import Flatten
from keras.layers.recurrent import LSTM
from keras.layers import merge,Concatenate
from keras.layers.merge import concatenate
from keras.models import Model
from keras.utils.vis_utils import plot_model
import tensorflow as tf
from keras.engine.topology import Layer,InputSpec
import numpy as np



In [41]:
class CylindricalPad(Layer):

    def __init__(self, n=1, **kwargs):
        super(CylindricalPad, self).__init__(**kwargs)
        self.n = n
        assert n > 0, 'n must be positive'
        
    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        super(CylindricalPad, self).build(input_shape)  
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0],
                    input_shape[1],
                    input_shape[2],
                    input_shape[3] + 2*self.n)
            
    def call(self, elementA, mask=None):
        element=elementA
        batchResultFlat=[]
        for batchElement in element:
            resultFlat=[]
            for cnnFilterImage in batchElement:
                firstColumns=cnnFilterImage[:,list(range(self.n))]
                result=np.concatenate((cnnFilterImage,np.array(firstColumns)),axis=1)
                lastColumnsReverse=cnnFilterImage[:,list(range(cnnFilterImage.shape[1]-self.n,cnnFilterImage.shape[1]))]
                result=np.concatenate((np.array(lastColumnsReverse),result),axis=1)
                resultFlat.append(result)
            batchResultFlat.append(np.array(resultFlat))
        newElement = np.array(batchResultFlat)
        return newElement

    def get_config(self):
        config = {'cropping': self.cropping}
        base_config = super(Cropping2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

print("CylindricalPad")

CylindricalPad


In [40]:

K.set_image_dim_ordering("th")

def transform(element,n):
    batchResultFlat=[]
    for batchElement in element:
        resultFlat=[]
        for cnnFilterImage in batchElement:
            print(cnnFilterImage)
            firstColumns=cnnFilterImage[:,list(range(n))]
            print( np.array(firstColumns))
            result=np.concatenate((cnnFilterImage,np.array(firstColumns)),axis=1)
            print(result)
            print(np.flipud(list(range(cnnFilterImage.shape[1]-n,cnnFilterImage.shape[1]))))
            #lastColumnsReverse=cnnFilterImage[:,np.flipud(list(range(cnnFilterImage.shape[1]-n,cnnFilterImage.shape[1])))]
            lastColumnsReverse=cnnFilterImage[:,list(range(cnnFilterImage.shape[1]-n,cnnFilterImage.shape[1]))]
            print(lastColumnsReverse)
            result=np.concatenate((np.array(lastColumnsReverse),result),axis=1)
            print(result)
            resultFlat.append(result)
        batchResultFlat.append(np.array(resultFlat))
    print(np.array(batchResultFlat))
    newElement = np.array(batchResultFlat)
    return newElement

def transformTest(element,n):
    if element.ndims > 0:
        return tf.map_fn(transformTest, inputs)
    else:
        return transform(inputs,n)
    return element

class RJPGNet:
    @staticmethod
    def build(timeSteps,variables,classes):
        #CONV=>POOL
        inputNet = Input(shape=(1,timeSteps,variables)) 
        conv1=Conv2D(20, (2,5), padding="same")(inputNet)
        
        #######################################################################
        conv1=CylindricalPad(n=2)(conv1)
        ################################# OR ##################################
        #conv1=Lambda(lambda element : transformTest(element,2))(conv1)
        #######################################################################
        
        conv2=Conv2D(5,(3,3), padding="same")(conv1)
        
        flatten=Flatten()(conv2)
                
        classificationLayer=Dense(classes)(flatten)
        classificationLayer=Activation("softmax")(classificationLayer)
        
        model=Model(inputNet,classificationLayer)
        return model
    

    
modeltest=RJPGNet.build(timeSteps=7,variables=5,classes=5)
modeltest.compile(optimizer="adadelta", loss="categorical_crossentropy",metrics=["accuracy"])
modeltest.summary() 

#plot_model(modeltest, to_file="model.png",show_shapes=True)

#plot_model(modeltest, to_file="model.png",show_shapes=True)
from IPython.display import SVG,display
from keras.utils.vis_utils import model_to_dot
modelSVG=SVG(model_to_dot(modeltest,show_shapes=True).create(prog='dot', format='svg'))
display(modelSVG)


TypeError: Tensor objects are not iterable when eager execution is not enabled. To iterate over this tensor use tf.map_fn.